In [25]:
# install the required packages
#%pip install pandas sentence-transformers ollama

In [26]:
#Installing mistral from Ollama - directly from terminal
#%pip install openpyxl
#%pip install chromadb

In [55]:
# Step 1: File Upload and Context Creation Module
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter

def create_context_from_excel(file_path):
    df = pd.read_excel(file_path)
    docs = []

    for _, row in df.iterrows():
        text = f"""
        TC ID: {row['TC ID']} TS Name: {row['Auto TS Name']} Quote Creation Source: {row['Quote Creation Source']}
        Copy Quote: {row['Copy Quote']} LOB: {row['LOB']} Sales Channel: {row['Sales Channel']} 
        Sales Channel: {row['Sales Channel']} Transaction Type: {row['Transaction Type']}
        Onboarding/Renewal: {row['Onboarding/Renewal']} Validate Customer: {row['Validate Customer']}
        Product Config: {row['Product Config']} Store Acceptance Type: {row['Store Acceptance Type']}
        Customer Type: {row['Customer Type']}
        Comments: {row['Comments']}
        """
        docs.append(text)

    # Convert to LangChain Documents
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    documents = text_splitter.create_documents(docs)

    # Create vector store
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(documents, embedding=embeddings, persist_directory="vectordb")
    vectordb.persist()
    return documents, vectordb

In [66]:
#Construct a formatted query and instructions to the LLM

def format_query(query_input):
    return (
        f"""Based on the following keywords:\n\n"{query_input}"\n\n"""
        "Search through the context and return ONLY the matching TC IDs.\n"
        "Do not return any value that is not attributed to TC ID and any extra explanation.\n"
        "Format the response as a comma-separated list of TC IDs like this:\n"
        "If you find multiple TC IDs, separate them with commas.\n"
        "do not respond with duplicate TC IDs.\n"
        
    )

In [67]:
# Step 2: Similarity Search Module
from langchain.chains import RetrievalQA
from langchain.llms import Ollama

def keyword_search(query, vectordb):
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})
    llm = Ollama(model="mistral")

    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    result = qa.run(query)
    return result

In [68]:
# Step 3: Jenkins Trigger Module
import requests

def trigger_jenkins(test_case_ids, job_name, jenkins_url, user, api_token):
    headers = {
        'Content-Type': 'application/json'
    }

    params = {
        'test_case_ids': ",".join(test_case_ids)
    }

    response = requests.post(
        f"{jenkins_url}/job/{job_name}/buildWithParameters",
        params=params,
        auth=(user, api_token),
        headers=headers
    )

    if response.status_code == 201:
        print("Jenkins build triggered successfully.")
    else:
        print(f"Failed to trigger Jenkins: {response.status_code} - {response.text}")

In [69]:
documents1, vectordb1 = create_context_from_excel("CPQCOE-AutomatedTestCases-Scope.xlsx")

In [70]:
print(documents1)

[Document(metadata={}, page_content='TC ID: CPQ-SILO-RT-01 TS Name: CPQ_RT_01_SILO_HGBUHWSW_CheckoutEligiblequote_multipleHWlines Quote Creation Source: CPQ\n        Copy Quote: No LOB: HGBU-HWSW Sales Channel: Direct \n        Sales Channel: Direct Transaction Type: Direct Buy\n        Onboarding/Renewal: Onboarding Validate Customer: Yes\n        Product Config: Micros HW Store Acceptance Type: Acceptance\n        Customer Type: Non PO HGBU\n        Comments: nan'), Document(metadata={}, page_content='TC ID: CPQ-SILO-RT-02 TS Name: CPQ_RT_02_SILO_HWSW_PTO_CustomerCRDdate Quote Creation Source: CPQ\n        Copy Quote: No LOB: HWSW Sales Channel: Direct \n        Sales Channel: Direct Transaction Type: Direct Buy\n        Onboarding/Renewal: Onboarding Validate Customer: Yes\n        Product Config: HW PTO, SW Store Acceptance Type: Acceptance\n        Customer Type: Non PO Enterprise\n        Comments: nan'), Document(metadata={}, page_content='TC ID: CPQ-SILO-RT-03 TS Name: CPQ_RT_0

In [72]:
query_input = "publicsector"
query = format_query(query_input)
response = keyword_search(query, vectordb1)
print("LLM Response:\n", response)

LLM Response:
  The provided context does not contain any "publicsector" keyword or related information, so there are no matching TC IDs to return.
